# **Installation**

In [48]:
!pip install audb


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
!pip install shap


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
!pip install opensmile


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
!pip install xgboost


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import audb
import audiofile
from xgboost import XGBClassifier
import opensmile
from sklearn.utils import shuffle
import random
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, log_loss, f1_score, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
%matplotlib inline


# Code

In [53]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

In [54]:
import os

def load_data_from_folds(directory, dir_aug):
    data = {"HS": [], "PD": [], "HS_aug": [], "PD_aug": []}
    
    for fold in range(1, 11):
        fold_path = os.path.join(directory, f"fold{fold}")
        hs_folder = os.path.join(fold_path, "HS")
        pd_folder = os.path.join(fold_path, "PD")
        
        hs_aug_folder = os.path.join(dir_aug, f"fold{fold}", "HS_aug")
        pd_aug_folder = os.path.join(dir_aug, f"fold{fold}", "PD_aug")

        hs_files = [os.path.join(hs_folder, f) for f in os.listdir(hs_folder) if f.endswith(".wav")]
        pd_files = [os.path.join(pd_folder, f) for f in os.listdir(pd_folder) if f.endswith(".wav")]

        hs_aug_files = [os.path.join(hs_aug_folder, f) for f in os.listdir(hs_aug_folder) if f.endswith(".wav")]
        pd_aug_files = [os.path.join(pd_aug_folder, f) for f in os.listdir(pd_aug_folder) if f.endswith(".wav")]

        data["HS"].append(hs_files)
        data["PD"].append(pd_files)
        data["HS_aug"].append(hs_aug_files)
        data["PD_aug"].append(pd_aug_files)
    
    return data


In [8]:
#Test
directory = r'C:\Users\Hp\Desktop\Lecture Files\DH604\Publication\data\datasets_16k'
dir_aug = r'C:\Users\Hp\Desktop\Lecture Files\DH604\Publication\data\datasets_aug_16k'
data_ = load_data_from_folds(directory, dir_aug)

In [9]:
data_['HS'][0]

['C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\HS\\B1ACNAGRER49F210320170916.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\HS\\B1ACNUTCOC40M230320171121.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\HS\\B2ACNAGRER49F210320170919.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\HS\\B2ACNUTCOC40M230320171123.wav']

In [10]:
data_['PD'][0]

['C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\PD\\B1AGNUTGOL52F100220171041.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\PD\\B1VSIOTLOP47M100220171328.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\PD\\B2AGNUTGOL52F100220171045.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_16k\\fold1\\PD\\B2VSIOTLOP47M100220171330.wav']

In [11]:
data_['HS_aug'][0]

['C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_aug_16k\\fold1\\HS_aug\\B1ACNAGRER49F210320170916.pink_noise.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_aug_16k\\fold1\\HS_aug\\B1ACNUTCOC40M230320171121.white_noise.wav']

In [12]:
data_['PD_aug'][9]

['C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_aug_16k\\fold10\\PD_aug\\B1ssacvhei61M1606161740.pink_noise.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\data\\datasets_aug_16k\\fold10\\PD_aug\\B1ubguot_t40M1606161755.white_noise.wav']

In [13]:
def process_audio_files(files, label, duration=10):
    results = []
    for file_path in files:
        if file_path.endswith(".wav"):
            try:
                signal, sampling_rate = audiofile.read(
                    file_path,
                    duration=duration,
                    always_2d=True
                )
                features = smile.process_signal(signal, sampling_rate)
                features.insert(0, 'file_name', file_path)
                features['Y'] = label
                results.append(features)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    
    df_results = pd.concat(results, ignore_index=True)
    # print(f"Done processing {len(results)} files.")
    return df_results


In [14]:
def drop_na(X, y):
    valid_indices = X.dropna(axis=1).index
    X = X.loc[valid_indices]
    y = y.loc[valid_indices]
    return X, y

In [15]:
def get_train_test_data(fold_number, data):
    test_hs = data["HS"][fold_number-1]
    test_pd = data["PD"][fold_number-1]

    # Combine the remaining folds for training
    #Original
    train_hs_ = [item for i, item in enumerate(data["HS"]) if i != fold_number-1]
    train_pd_ = [item for i, item in enumerate(data["PD"]) if i != fold_number-1]
    # print("train_hs_:", len(train_hs_))
    # print("train_pd_:", len(train_pd_))

    #Aug
    train_hs_aug = [item for i, item in enumerate(data["HS_aug"]) if i != fold_number-1]
    train_pd_aug = [item for i, item in enumerate(data["PD_aug"]) if i != fold_number-1]
    # print("train_hs_aug:", len(train_hs_aug))
    # print("train_pd_aug:", len(train_pd_aug))
    

    train_hs = train_hs_ + train_hs_aug
    train_pd = train_pd_ + train_pd_aug


    # Flatten lists for training
    train_hs = [item for sublist in train_hs for item in sublist]
    # print("train_hs:", len(train_hs))
    train_pd = [item for sublist in train_pd for item in sublist]
    # print("train_pd:", len(train_pd))

    # Process the files
    X_train_hs = process_audio_files(train_hs, label=0)
    X_train_pd = process_audio_files(train_pd, label=1)
    
    X_test_hs = process_audio_files(test_hs, label=0)
    X_test_pd = process_audio_files(test_pd, label=1)

    # Combine HS and PD for train and test
    X_train = pd.concat([X_train_hs, X_train_pd], ignore_index=True)
    X_test = pd.concat([X_test_hs, X_test_pd], ignore_index=True)


    y_train = X_train["Y"]
    X_train.drop(columns=["Y",'file_name'], inplace=True)

    y_test = X_test["Y"]
    X_test.drop(columns=["Y",'file_name'], inplace=True)

    # Shuffle the train and test sets
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    X_test, y_test = shuffle(X_test, y_test, random_state=42)


    # print("X_train shape:", X_train.shape)
    # print("y_train shape:", y_train.shape)
    # print("X_test shape:", X_test.shape)
    # print("y_test shape:", y_test.shape)

    # Drop NA
    X_train, y_train = drop_na(X_train, y_train)
    X_test, y_test = drop_na(X_test, y_test)

    # Scaling the data
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    return X_train, y_train, X_test, y_test


In [16]:
X_train, y_train, X_test, y_test = get_train_test_data(10, data_)

In [17]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (105, 6373)
y_train shape: (105,)
X_test shape: (8, 6373)
y_test shape: (8,)


In [18]:
X_train.head(2)

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
30,-0.753220,1.956989,-1.562788,-0.506750,-0.602905,-0.688103,-0.525029,-0.560605,-0.571132,-0.482686,...,-0.560261,-0.882924,-0.319443,-0.30667,-0.961923,0.376740,-0.027397,0.196108,0.190301,-0.464967
65,-0.146795,1.856863,1.774561,-0.231611,0.061182,0.189163,1.781618,0.623439,1.171746,-0.486467,...,-1.090558,-0.645290,-0.667193,-0.68381,1.056385,0.380269,-0.607664,-0.022428,-0.959468,-0.319956


In [19]:
y_train.head()

30    0
65    1
64    1
53    1
45    0
Name: Y, dtype: int64

In [20]:
y_test.value_counts()

Y
0    4
1    4
Name: count, dtype: int64

In [72]:
def create_model(model_type):
    if model_type == 'random_forest':
        # model = RandomForestClassifier(random_state=42)
        model = RandomForestClassifier(max_depth=3, max_features='log2', min_samples_leaf=2, min_samples_split=5, n_estimators=100, random_state=42)

    elif model_type == 'logistic_regression':
        # model = LogisticRegression(random_state=42)
        model =LogisticRegression(C=0.01, max_iter=1000, penalty='elasticnet', solver='saga', l1_ratio = 0, random_state=42)

    elif model_type == 'svm':
        # model = SVC(random_state=42, probability=True)
        model = SVC(C=0.1 ,class_weight=None , gamma=0.01, kernel='rbf', shrinking=True, probability=True, random_state=42)

    elif model_type == 'xgboost':
        # model = XGBClassifier(random_state=42)
        model = XGBClassifier(colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=2, min_child_weight=1, n_estimators=100, reg_alpha=0.1, reg_lambda=1, subsample=0.8)

    else:
        raise ValueError("Invalid model type. Choose from 'random_forest', 'logistic_regression', 'svm', or 'xgboost'.")
    return model

In [56]:
def evaluate_model(X_train, y_train, X_test, y_test, model_type):

    if model_type == 'xgboost':
        for X in [X_train, X_test]:
            X.columns = [col.replace('[', '(').replace(']', ')').replace('<', '_less_than_') for col in X.columns]

    model = create_model(model_type)
    
    model.fit(X_train, y_train)
    
    # Get predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics
    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    
    roc_auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    roc_auc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    f1_train = f1_score(y_train, y_train_pred)
    f1_test = f1_score(y_test, y_test_pred)
    
    precision_train = precision_score(y_train, y_train_pred)
    precision_test = precision_score(y_test, y_test_pred)
    
    recall_train = recall_score(y_train, y_train_pred)
    recall_test = recall_score(y_test, y_test_pred)
    
    train_loss = log_loss(y_train, model.predict_proba(X_train)[:, 1])
    test_loss = log_loss(y_test, model.predict_proba(X_test)[:, 1])

    return {
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "roc_auc_train": roc_auc_train,
        "roc_auc_test": roc_auc_test,
        "f1_train": f1_train,
        "f1_test": f1_test,
        "precision_train": precision_train,
        "precision_test": precision_test,
        "recall_train": recall_train,
        "recall_test": recall_test,
        "train_loss": train_loss,
        "test_loss": test_loss
    }


In [58]:
# MIQ

selected_features = [
    "audspecRasta_lengthL1norm_sma_upleveltime75",
    "audSpec_Rfilt_sma[6]_percentile1.0",
    "audSpec_Rfilt_sma[13]_quartile2",
    "audSpec_Rfilt_sma[21]_quartile2",
    "pcm_fftMag_spectralRollOff25.0_sma_iqr1-2",
    "pcm_fftMag_spectralRollOff90.0_sma_quartile1",
    "pcm_fftMag_spectralCentroid_sma_lpc2",
    "mfcc_sma[2]_leftctime",
    "mfcc_sma[4]_minSegLen",
    "mfcc_sma[8]_meanSegLen",
    "audSpec_Rfilt_sma_de[6]_minSegLen",
    "audSpec_Rfilt_sma_de[8]_minSegLen",
    "audSpec_Rfilt_sma_de[12]_minSegLen",
    "audSpec_Rfilt_sma_de[21]_minSegLen",
    "pcm_fftMag_spectralRollOff90.0_sma_de_minSegLen",
    "mfcc_sma_de[3]_minSegLen",
    "mfcc_sma_de[7]_upleveltime25",
    "voicingFinalUnclipped_sma_range",
    "voicingFinalUnclipped_sma_upleveltime90",
    "jitterDDP_sma_de_quartile2",
    "jitterDDP_sma_de_upleveltime75",
    "jitterDDP_sma_de_lpc0",
    "audSpec_Rfilt_sma[7]_qregerrQ",
    "pcm_fftMag_spectralSlope_sma_qregc3",
    "pcm_fftMag_psySharpness_sma_flatness",
    "mfcc_sma[5]_peakMeanRel",
    "mfcc_sma[6]_peakMeanRel",
    "mfcc_sma[7]_peakMeanRel",
    "audSpec_Rfilt_sma_de[1]_peakRangeRel",
    "pcm_fftMag_spectralEntropy_sma_de_stddevFallingSlope"
]

In [59]:
def main(directory, dir_aug, model_type, feature_list=True):
    data = load_data_from_folds(directory, dir_aug)
    
    metrics = []
    
    for fold_number in range(1, 11):
        print(f"Evaluating fold {fold_number}...")
        
        X_train, y_train, X_test, y_test = get_train_test_data(fold_number, data)
        
        if feature_list:
            X_train = X_train[selected_features]
            X_test = X_test[selected_features]
        
        print("X_train shape:", X_train.shape)
        print("y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape)
        print("y_test shape:", y_test.shape)
        
        fold_metrics = evaluate_model(X_train, y_train, X_test, y_test, model_type)
        metrics.append(fold_metrics)
        
        print(f"Metrics for fold {fold_number}: {fold_metrics}")
        print(" ")
    
    # Calculate average and standard deviation
    avg_metrics = {metric: np.mean([fold[metric] for fold in metrics]) for metric in metrics[0]}
    std_metrics = {metric: np.std([fold[metric] for fold in metrics]) for metric in metrics[0]}
    
    print("\nAverage Metrics:")
    print(avg_metrics)
    
    print("\nStandard Deviation of Metrics:")
    print(std_metrics)


In [60]:
print(directory)
print(dir_aug)

C:\Users\Hp\Desktop\Lecture Files\DH604\Publication\data\datasets_16k
C:\Users\Hp\Desktop\Lecture Files\DH604\Publication\data\datasets_aug_16k


In [61]:
# # Call the main function with your model type
# main(directory, dir_aug, model_type="logistic_regression", feature_list = True)

In [62]:
# # Call the main function with your model type
# main(directory, dir_aug, model_type="svm", feature_list = True)

In [70]:
# # Call the main function with your model type
# main(directory, dir_aug, model_type="random_forest", feature_list = True)

In [73]:
# Call the main function with your model type
main(directory, dir_aug, model_type="xgboost", feature_list = True)

Evaluating fold 1...
X_train shape: (105, 30)
y_train shape: (105,)
X_test shape: (8, 30)
y_test shape: (8,)
Metrics for fold 1: {'accuracy_train': 1.0, 'accuracy_test': 1.0, 'roc_auc_train': 1.0, 'roc_auc_test': 1.0, 'f1_train': 1.0, 'f1_test': 1.0, 'precision_train': 1.0, 'precision_test': 1.0, 'recall_train': 1.0, 'recall_test': 1.0, 'train_loss': 0.28474317598756954, 'test_loss': 0.2851279545512537}
 
Evaluating fold 2...
X_train shape: (105, 30)
y_train shape: (105,)
X_test shape: (8, 30)
y_test shape: (8,)
Metrics for fold 2: {'accuracy_train': 1.0, 'accuracy_test': 1.0, 'roc_auc_train': 1.0, 'roc_auc_test': 1.0, 'f1_train': 1.0, 'f1_test': 1.0, 'precision_train': 1.0, 'precision_test': 1.0, 'recall_train': 1.0, 'recall_test': 1.0, 'train_loss': 0.2847666549928742, 'test_loss': 0.24566216036019312}
 
Evaluating fold 3...
X_train shape: (105, 30)
y_train shape: (105,)
X_test shape: (8, 30)
y_test shape: (8,)
Metrics for fold 3: {'accuracy_train': 1.0, 'accuracy_test': 1.0, 'roc_au